Problema de Selección

Contexto
Un determinado puerto marítimo tiene 3 zonas de descarga C1, C2 y C3; 2 zonas de 
carga S1, S2; 2 depósitos para almacenaje D1, D2 y 3 terminales T1, T2 y T3. Cada 
instante de tiempo se reciben y solicitan contenedores vacíos, que se redistribuyen 
para satisfacer la oferta y la demanda de comercio ultramarino. 

Datos
Las capacidades máximas de almacenaje tanto en terminales como en depósitos son 
de 20 contenedores, mientras que los costes de almacenaje por contenedor son de 20 
para la terminal T1, 30 para la terminal T2, 25 para la T3, 22 para D1 y 28 para D2. 
En un cierto instante de tiempo, cada depósito y cada terminal, excepto la T2, tienen 
un contenedor vacío en stock. Adicionalmente, en ese mismo instante, cada zona de 
carga y la terminal T2 reclaman cada una un contenedor vacío para importación ultramarítima, así como cada zona de descarga provee de un contenedor vacío proveniente 
de comercio de ultra mar. Los costes de transporte de contenedores vienen dados por 
las siguientes matrices, que contemplan los únicos desplazamientos permitidos (que 
son a su vez asimétricos): 
De zona de descarga (fila) a terminal (columna): 
[[14,15,12],
[15,17,12],
[20,19,17]]
De zona de descarga (fila) a depósito (columna): 
[[12,10],
[19,12],
[16,18]]
De zona de terminal (fila) a zona de carga (columna):
[[14,15],
[18,17],
[12,20]]
De zona de deposito (fila) a zona de carga(columna): 
[[11,17],
[12,14]]
De zona de terminal (fila) a terminal (columna):
[[0,15,18],
[15,0,12],
[17,16,0]]
De zona de depósito (fila) a depósito (columna): 
[[0,18],
[16,0]]
De zona de terminal (fila) a depósito (columna):
[[16,15],
[14,17],
[13,16]]
De zona de depósito (fila) a terminal (columna):
[[18,16,17],
[14,17,18]]

Objetivo
Plantear el problema de optimización para reducir costes y satisfacer la oferta y 
demanda de contenedores vacíos en ese instante (pista: hay restricciones que quizá no 
sean tan evidentes de formular). El resultado ideal debería de ser los movimientos de 
contenedores que cada solución sugiere, si la solución es capaz de satisfacer la 
demanda y qué coste total tiene asociado. Con esto resultados haremos un pequeño 
análisis cuando nos juntemos de nuevo

In [2]:
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel

#Set up del escenario
origen = [0,1,2,3,4,5,6,7,8,9] #Ubicación inicial
destino = [0,1,2,3,4,5,6,7,8,9] #Ubicación final
costes = [ [0,0,0,34,38,34,45,37,0,0],
           [0,0,0,41,40,35,47,37,0,0],
           [0,0,0,38,46,40,49,42,0,0],
           [0,0,0,22,46,38,46,42,11,17],
           [0,0,0,38,28,34,47,43,12,14],
           [0,0,0,38,43,20,45,43,12,10],
           [0,0,0,36,45,35,30,37,19,12],
           [0,0,0,35,44,37,46,25,16,18],
           [0,0,0,22,28,20,30,25,0,0],
           [0,0,0,22,28,20,30,25,0,0],] 

#Una variable para cada posible ubicación final de cada ubicación inicial
x = [[f'x{o}{d}' for d in destino] for o in origen]

#Iniciar BQM
bqm=BinaryQuadraticModel('BINARY')

#Objetivo
for o in origen:
    for d in destino:
        bqm.add_variable(x[o][d], costes[o][d]) 

#Restricciones
#a) Capacidad máxima de almacenamiento (TRIVIAL)
destinos_almacenaje = [3,4,5,6,7]
for d in destinos_almacenaje:
    r1 = [(x[o][d],1) for o in origen]
    bqm.add_linear_inequality_constraint(r1,
            ub = 20,
            lagrange_multiplier = 1,
            label = 'r1_capacidad_'+str(d))

#b) Solicitud de contenedores (demanda)
destinos_carga = [8,9]
for d in destinos_carga:
    r2 = [(x[o][d],1) for o in origen]
    bqm.add_linear_equality_constraint(r2,
            constant = 1,
            lagrange_multiplier = 2)
            
destinos_demanda = [6]
for d in destinos_demanda:
    r3 = [(x[o][d],1) for o in origen]
    bqm.add_linear_inequality_constraint(r3,
            lb = 1,
            ub = 20,
            lagrange_multiplier = 2,
            label = 'r3_demanda_'+str(d)) 
    
#c) Provisión de contenedores (oferta)
origen_descarga = [0,1,2]
for o in origen_descarga:
    r4 = [(x[o][d],1) for d in destino]
    bqm.add_linear_equality_constraint(r4,
            constant = 1,
            lagrange_multiplier = 1)

#d) Condición inicial
origen_contenedores = [3,4,5,7]
for o in origen_contenedores:
    r5 = [(x[o][d],1) for d in destino]
    bqm.add_linear_equality_constraint(r5,
            constant = 1,
            lagrange_multiplier = 1)
    
#e) Prohibición de permanencia en descargas
destinos_descarga = [0,1,2]
for d in destinos_descarga:
    r6 = [(x[o][d],1) for o in origen]
    bqm.add_linear_equality_constraint(r6,
            constant = 0,
            lagrange_multiplier = 1)  

#e) Ubicaciones sin contenedores inicialmente
origen_vacio = [6,8,9]
for o in origen_vacio:
    r7 = [(x[o][d],1) for d in destino]
    bqm.add_linear_equality_constraint(r7,
            constant = 0,
            lagrange_multiplier = 1)  
    
#f) Imposibilidad de tránsito Descarga-Carga
for o in origen_descarga:
    r8 = [(x[o][d],1) for d in destinos_carga]
    bqm.add_linear_equality_constraint(r8,
            constant = 0,
            lagrange_multiplier = 1)  

#g) número de trayectos
r9 = [(x[o][d], 1) for o in origen for d in destino] 
bqm.add_linear_equality_constraint(r9,
    constant=7,  
    lagrange_multiplier=10)      
         

C:\Users\crist\AppData\Roaming\Python\Python313\site-packages\dimod\binary\binary_quadratic_model.py:773: UserWarning: Did not add constraint r1_capacidad_3. This constraint is feasible with any value for state variables.
  warnings.warn(
C:\Users\crist\AppData\Roaming\Python\Python313\site-packages\dimod\binary\binary_quadratic_model.py:773: UserWarning: Did not add constraint r1_capacidad_4. This constraint is feasible with any value for state variables.
  warnings.warn(
C:\Users\crist\AppData\Roaming\Python\Python313\site-packages\dimod\binary\binary_quadratic_model.py:773: UserWarning: Did not add constraint r1_capacidad_5. This constraint is feasible with any value for state variables.
  warnings.warn(
C:\Users\crist\AppData\Roaming\Python\Python313\site-packages\dimod\binary\binary_quadratic_model.py:773: UserWarning: Did not add constraint r1_capacidad_6. This constraint is feasible with any value for state variables.
  warnings.warn(
C:\Users\crist\AppData\Roaming\Python\Python

In [3]:
sampler = EmbeddingComposite(DWaveSampler(token = "DEV-5042821ad2e2d999ab962237ae05315c378993b2"))
sampleset = sampler.sample(bqm, num_reads=100)

sample= sampleset.first.sample
coste_total = 0

print("\n\t0\t1\t2\t3\t4\t5\t6\t7\t8\t9")
for o in origen:
    printout =str(o)
    for d in range (len(destino)):
        printout += "\t" + str(sample[x[o][d]])
        coste_total += sample[x[o][d]]*costes[o][d]
    print (printout)

print("Coste total:\t", coste_total, "\n") 


	0	1	2	3	4	5	6	7	8	9
0	0	0	0	0	0	0	0	0	0	0
1	0	0	0	0	0	0	0	0	0	0
2	0	0	1	0	0	0	0	0	0	0
3	0	0	0	0	0	0	0	0	0	0
4	0	1	0	0	0	1	0	0	0	0
5	0	0	1	0	0	1	0	0	0	0
6	0	0	0	0	0	0	0	0	0	0
7	0	0	0	0	0	0	0	1	0	0
8	0	0	0	0	0	0	0	1	1	0
9	0	0	0	0	0	0	0	0	0	0
Coste total:	 104 

